## Forecasting for Monitor

This notebook walks through building charts used in the Monitor application.

To run this notebook, start Jupyter as follows from the Django project root: <br>
(https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

`python manage.py shell_plus --notebook`

<br>
NOTE:  You will need to changed the kernel from menu: `Kernel` > `Change kernel` > `Django Shell-Plus`

In [8]:
import os
import logging
import pandas as pd
from pandas import Grouper
from pygam import LinearGAM, s


In [9]:


try:
    from django.db import models
    from traffic_monitor.models.model_logentry import LogEntry
    

except Exception as e:
    print("NOT LOADED:  start notebook with:\n")
    print("\tpython manage.py shell_plus --notebook")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper, ColorBar, FixedTicker, NumeralTickFormatter

from bokeh.models import HoverTool, WheelZoomTool, ResetTool
from bokeh.models import Legend, LegendItem
from bokeh.plotting import figure
from bokeh.palettes import brewer, RdYlGn11
from bokeh.embed import components
from bokeh.embed import json_item
from bokeh.models import ColumnDataSource, HTMLTemplateFormatter, TableColumn, DataTable, HoverTool, \
    DatetimeTickFormatter, Line, FactorRange
from bokeh.plotting import figure
from django.http import JsonResponse
output_notebook()

Loading BokehJS ...

In [10]:
logger = logging.getLogger("model_logger")
logger.setLevel(logging.INFO)

In [11]:
rs = LogEntry.objects.all().values()

In [19]:
raw_df = pd.DataFrame(rs)
raw_df

,key,time_stamp,monitor_id,class_name,count
0,33,2020-08-28 19:59:46.557975+00:00,MyMonitor,car,3.308
1,34,2020-08-28 19:59:46.557975+00:00,MyMonitor,person,0.231
2,35,2020-08-28 20:00:46.711236+00:00,MyMonitor,car,3.417
3,36,2020-08-28 20:00:46.711236+00:00,MyMonitor,person,0.833
4,37,2020-08-28 20:01:46.725153+00:00,MyMonitor,car,3.000
...,...,...,...,...,...
25774,26085,2020-10-08 00:29:18.427580+00:00,MyMonitor,bus,0.071
25775,26086,2020-10-08 00:34:18.977291+00:00,MyMonitor,truck,0.138
25776,26087,2020-10-08 00:34:18.977291+00:00,MyMonitor,car,4.724
25777,26088,2020-10-08 00:34:18.977291+00:00,MyMonitor,person,2.897


In [18]:
# Create columns for day of week

In [29]:
raw_df['weekday'] = raw_df.time_stamp.apply(lambda s: s.weekday())

In [56]:
# Categorize timestamps into 15 minute segments: 
# 15(0-14), 30(15-29), 45(30-44), 60(45-59)

In [101]:
raw_df['qtr_hour'] = raw_df.time_stamp.apply(lambda s : int(s.minute / 15 + 1 % 5) * 15)
raw_df['hour'] = raw_df.time_stamp.apply(lambda s: s.hour)

In [102]:
raw_df.head()

,key,time_stamp,monitor_id,class_name,count,weekday,qtr_hour,hour
0,33,2020-08-28 19:59:46.557975+00:00,MyMonitor,car,3.308,4,60,19
1,34,2020-08-28 19:59:46.557975+00:00,MyMonitor,person,0.231,4,60,19
2,35,2020-08-28 20:00:46.711236+00:00,MyMonitor,car,3.417,4,15,20
3,36,2020-08-28 20:00:46.711236+00:00,MyMonitor,person,0.833,4,15,20
4,37,2020-08-28 20:01:46.725153+00:00,MyMonitor,car,3.000,4,15,20
